<h1>copyright statement</h1>

Copyright 2018 Mahmoud Abdelrazek, Ibrahim Alhadidi.<br>
Permission is hereby granted, free of charge,<br>
to any person obtaining a copy of this software <br>
and associated documentation files (the “Software”),<br>
to deal in the Software without restriction,<br>
including without limitation the rights to use, copy, modify,<br>
merge, publish, distribute, sublicense, and/or sell copies of the Software,<br>
and to permit persons to whom the Software is furnished to do so,<br>
subject to the following conditions:<br>
The above copyright notice and this permission notice shall be included<br>
in all copies or substantial portions of the Software.<br>
THE SOFTWARE IS PROVIDED “AS IS”,<br>
WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED,<br>
INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,<br>
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.<br>
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM,<br>
DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE,<br>
ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE<br>
OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.<br>

THIS STATEMENT APPLIES ONLY TO THE CODE IN THIS NOTEBOOK!
THE RESOURCES USED (LIBRARIES, FILES, ... ) ARE NOT CREATED NOT OWNED BY US AND THEREFORE THE USE OF THEM FOLLOWS THEIR OWN COPYRIGHT RULES

SOME OF THE CODE WAS PROVIDED TO US AS PART OF UCL GEOSPATIAL PROGRAMMING MODULE FOR THE PURPOSE OF COMPLETING THIS ASSIGNMENT 

In [ ]:
%matplotlib inline

Import the necessary libraries.

In [ ]:
import cartopy.crs as ccrs
import geopandas as gpd
from IPython.display import clear_output, display, Markdown 
from IPython.core.display import HTML
import json
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib import cm
import networkx as nx
import numpy as np
import rasterio
from rasterio.transform import xy, rowcol, from_bounds
from rasterio import features
from rasterio.windows import Window
from rtree import index
from shapely.geometry import Point, LineString, box
import sys
import time

Defining a function to prompt the user to input the current location.<br> The function also checks the input of the user <br> to ensure it is within the bounds of the area used for calcuations  

In [ ]:
def input_start_point():
    while True:
        x = input('Please input Easting, value shoud be between 425,000 and 470,000: ')
        try:
            x = float(x)
        except:
            print('Please Enter a Number')
            continue
        if not (425000 <= x <= 470000):
            print("Please Enter Valid Easting, value shoud be\
                   between 425,000 and 470,000")
            continue
        else:
            break

    while True:
        y = input('Please input Northing, value shoud be between 75,000 and 100,000: ')
        try:
            y = float(y)
        except:
            print('Please Enter a Number, value shoud be\
                   between 75,000 and 100,000')
            continue
        if not (75000 <= y <= 100000):
            print("Please Enter Valid Northing, value shoud be\
                   between 75,000 and 100,000")
            continue
        else:
            break

    return x, y

Prompting the user to insert the current location using <br> the function which was deined in the pervious cell <br>
Also defining then calculating the bounds based on the maximum buffer distance.

In [ ]:
point = input_start_point()
point_obj = Point(point)
buffer_distance = 5000
west_bound, east_bound = point[0]-buffer_distance, point[0]+buffer_distance
south_bound, north_bound = point[1]-buffer_distance, point[1]+buffer_distance

Load the legend image, start and end points symbols.

In [ ]:
# Read Legend Image
clear_output()
display(HTML('<h3><span style="color:blue"> loading the necessary files ...\
              </span></h3>'))
legend_img = mpimg.imread('images/legend.png')
geotag_img = mpimg.imread('images/geotag.png')
geotag_destination_img = mpimg.imread('images/geotag_destination.png')

Loading the ITN file <br>
and extracting the road_nodes data

In [ ]:
# Read Network
solent_itn_json = 'material/itn/solent_itn.json'
with open(solent_itn_json) as f:
    solent_itn = json.load(f)

roads = solent_itn['road']
road_links = solent_itn['roadlinks']
road_nodes = solent_itn['roadnodes']

Defining a function to create a raster window based on the start point as a central point and the buffer distance.

In [ ]:
def create_window(point, raster_layer, buffer_distance=5000):
    row_offset, col_offset = raster_layer.index(west_bound, north_bound)
    row_opposite, col_opposite = raster_layer.index(east_bound, south_bound)
    width = col_opposite - col_offset
    height = row_opposite - row_offset
    window = Window(col_offset, row_offset, width, height)
    return window

**Advanced point:** a function to add "padding" around the heights raster <br> to accommodate for the remote points within 5km from the edges of the raster.

In [ ]:
def fill_array(raster, array, point_obj,
               resolution, buffer_distance):

    expected_size = int(buffer_distance/raster.transform[0] * 2)

    if raster.shape == (expected_size, expected_size):
        return array
    else:
        horizontal_diff = expected_size - array.shape[1]
        horizontal_fill = np.empty((array.shape[0], horizontal_diff))

        vertical_diff = expected_size - array.shape[0]
        vertical_fill = np.empty((vertical_diff, expected_size))

    bounds = raster.bounds

    if point_obj.x - bounds.left < buffer_distance:
        array = np.hstack([horizontal_fill, array])
    elif bounds.right - point_obj.x < buffer_distance:
        array = np.hstack([array, horizontal_fill])

    if point_obj.y - bounds.bottom < buffer_distance:
        array = np.vstack([array, vertical_fill])
    elif bounds.top - point_obj.y < buffer_distance:
        array = np.vstack([vertical_fill, array])

    return array

Load the heights raster as a numpy array <br>
using on the window created by the function in the previous cell, <br>
and defining the affine transform.

In [ ]:
# Read Elevation Raster Data
elevation_raster_path = 'material/elevation/SZ.asc'
elevation_raster = rasterio.open(elevation_raster_path)
resolution = elevation_raster.transform[0]

elevation_raster_box = box(*list(elevation_raster.bounds))
distance_from_bounds = elevation_raster_box.exterior.distance(point_obj)

window_elevation = create_window(point, elevation_raster, buffer_distance)
heights_array = elevation_raster.read(1, window=window_elevation)
heights_array = fill_array(elevation_raster, heights_array,
                           point_obj, resolution, buffer_distance)

# elevation_raster.transform[0] is the resolution in meters
# it's more generic way, better than hard coding the raster resolution
number_of_columns = (east_bound-west_bound)/elevation_raster.transform[0]
number_of_rows = (north_bound-south_bound)/elevation_raster.transform[0]
heights_affine_for_window = from_bounds(west_bound, south_bound,
                                        east_bound, north_bound,
                                        number_of_columns,
                                        number_of_rows)

Load the island raster and applying the embedded colormap

In [ ]:
# Read Island Raster Data
island_raster_path = 'material/os50k/raster-50k_2724246.tif'
island_raster = rasterio.open(island_raster_path)

window_island = create_window(point, island_raster, buffer_distance)
island_raster_array = island_raster.read(1, window=window_island)
island_raster_array = fill_array(island_raster, island_raster_array,
                                 point_obj, resolution, buffer_distance)

palette = np.array([value for key, value in island_raster.colormap(1).items()])
island_raster_image = palette[island_raster_array.astype(int)]


Creating a rasterized version of the point buffer to clip the heights array based on it. 

In [ ]:
clear_output()
display(HTML('<h3><span style="color:blue"> Performing calculations ...\
              </span></h3>'))
# Point Buffer
point_buffer = point_obj.buffer(buffer_distance)

# Rasterize Buffer
number_of_columns = int((east_bound-west_bound) / resolution)
number_of_rows = int((north_bound-south_bound) / resolution)
array_shape = (number_of_rows, number_of_columns)
affine_transform = from_bounds(west_bound, south_bound,
                               east_bound, north_bound,
                               number_of_columns, number_of_rows)
point_buffer_mask = features.rasterize([(point_buffer, 1)],
                                       out_shape=array_shape,
                                       transform=affine_transform)
point_buffer_mask = ~point_buffer_mask.astype(bool)

# Clip Heights with point buffer
heights_array_clipped = heights_array.copy()
heights_array_clipped[point_buffer_mask] = np.nan

Get the coordinates of the highest point within the buffer area.

In [ ]:
# Get Highest Point
row, col = np.unravel_index(np.nanargmax(heights_array_clipped),
                            heights_array_clipped.shape)
x_highest, y_highest = xy(affine_transform, row, col)
highest_point = (x_highest, y_highest)

Creating R tree index of all nodes within the buffer area, <br> to find out the closest node to the starting point and the highest point. <br>
Also checking if there are no transportation nodes <br>
within 5km radius of the entered point.  


In [ ]:
# Create R tree index and find nearest point
idx = index.Index()
for i, (fid, coords) in enumerate(road_nodes.items()):
    (x, y) = coords['coords']
    node_as_point = Point(x, y)
    if not point_buffer.contains(node_as_point):
        continue
    idx.insert(i, (x, y), fid)

try:
    nearest_origin_point_fid = list(idx.nearest(point, num_results=1,
                                                objects=True))[0].object
except IndexError as e:
    clear_output()
    display(HTML("<h1><center> It seems that there are no transportation nodes\
                  near you </center></h1>"))
    display(HTML("<h1><center> Please, relocate and refresh the page \
                  </center></h1>"))
    start_point = ("<h2><center>" + "The point you entered is at " +
                   str(int(point[0])) + ", " + str(int(point[1])) +
                   "</center></h2>")
    display(HTML(start_point))
    while True:
        pass

nearest_target_point_fid = list(idx.nearest(highest_point,
                                num_results=1, objects=True))[0].object
nearest_target_coords = road_nodes[nearest_target_point_fid]['coords']
nearest_origin_coords = road_nodes[nearest_origin_point_fid]['coords']

Defining a function to implement the naismith's rule, which calcualtes the additional time in minutes for each road link

In [ ]:
def calcualte_additional_time(coords, heights_array, affine_transform):
    total_climb = 0
    for i, point in enumerate(coords):
        x, y = point
        if i == 0:
            last_height = heights_array[rowcol(affine_transform, x, y)]
        else:
            height = heights_array[rowcol(affine_transform, x, y)]
            if height > last_height:
                total_climb += height-last_height

            last_height = height
    additional_time = total_climb/10
    return additional_time  # in minutes

A function to check if a link is inside a polygon, will be used to make sure that all link points lies within the buffer area.

In [ ]:
def is_link_inside_polygon(coords, polygon):
    inside = True
    for x_y in coords:
        point_obj = Point(x_y)
        if not polygon.contains(point_obj):
            inside = False
            break
    return inside

Create the network of edges connecting the nodes, and calculating the time required to go through each edge both ways. 

In [ ]:
# Create the network
walking_speed = 5
km_to_meters = 1000
hour_to_minutes = 60
G = nx.DiGraph()
for link in road_links:
    length = road_links[link]['length']
    coords = road_links[link]['coords']

    if not is_link_inside_polygon(coords, point_buffer):
        continue

    base_time = length/(walking_speed * km_to_meters) * hour_to_minutes

    # Forward
    additional_time = calcualte_additional_time(coords, heights_array,
                                                heights_affine_for_window)
    total_time = base_time+additional_time
    G.add_edge(road_links[link]['start'],
               road_links[link]['end'],
               fid=link,
               length=road_links[link]['length'],
               time=total_time)

    # Backward
    additional_time = calcualte_additional_time(reversed(coords),
                                                heights_array,
                                                heights_affine_for_window)
    total_time = base_time+additional_time
    G.add_edge(road_links[link]['end'],
               road_links[link]['start'],
               fid=link,
               length=road_links[link]['length'],
               time=total_time)

A function to convert the shortest path to a GeoDataFrame for plotting purposes

In [ ]:
def path_to_gpd(path, G, road_links):
    links = []
    geom = []
    for n, each in enumerate(path):
        if n==0:
            pass
        else:
            link = G.edges[last, each]['fid']
            links.append(link)
            geom.append(LineString(road_links[link]['coords']))
        last = each

    path_gpd = gpd.GeoDataFrame({'fid': links, 'geometry': geom})
    return path_gpd

Finding the shortest path based on the time, and converting the path to a GeoDataFrame for plotting.

In [ ]:
# Find Shortest Path and convert it to GeoDataFrame
try:
    shortest_path = nx.dijkstra_path(G,
                                     source=nearest_origin_point_fid,
                                     target=nearest_target_point_fid,
                                     weight='time')
except nx.NetworkXNoPath:
    clear_output()
    display(HTML("<h1><center> It seems that the only way is outside\
                  the 5 Km radius</center></h1>"))
    display(HTML("<h1><center> Please, relocate and refresh the page \
                  </center></h1>"))
    start_point = ("<h2><center>" + "The point you entered is at " +
                   str(int(point[0])) + ", " + str(int(point[1])) +
                   "</center></h2>")
    display(HTML(start_point))
    while True:
        pass
shortest_path_gdf = path_to_gpd(shortest_path, G, road_links)

Adding the (off-road) lines which represent at the begining the route from the starting poitn to the first node in the network and from the last node in the network to the highest point. <br> <br>
**Creativity point:** handling a special case of the path including only one point (i.e) if the start and the end point are very close to each other that the network would only include a single node represting both the first and the last node. 

In [ ]:
# Adding off road lines
if not shortest_path_gdf.empty:
    start_node_coords = shortest_path_gdf.iloc[0].geometry.coords[0]
    end_node_coords = shortest_path_gdf.iloc[-1].geometry.coords[-1]
    start_line_obj = LineString([point, start_node_coords])
    end_line_obj = LineString([end_node_coords, highest_point])
    off_road_route = gpd.GeoDataFrame({'fid': ['starting_line', 'ending_line'],
                                       'geometry': [start_line_obj,
                                                    end_line_obj]})
else:
    direct_line_obj = LineString([point, highest_point])
    off_road_route = gpd.GeoDataFrame({'fid': ['direct_line'],
                                       'geometry': [direct_line_obj]})    

Defining the extents required for all rasters to be ploted

In [ ]:
# Defining all needed extents
display_extents = (west_bound, east_bound, south_bound, north_bound)
display(HTML(f'<h3>{display_extents}</h3>'))
legend_extents = (west_bound+50, west_bound+721*3+50,
                  south_bound+50, south_bound+621*3+50)
geotag_extents = (point[0]-177, point[0]+177,
                  point[1], point[1]+512)

geotag_destination_extents = (highest_point[0]-177, highest_point[0]+177,
                              highest_point[1], highest_point[1]+512)


# Plotting
clear_output()
display(HTML('<h3><span style="color:blue"> Exporting graphics ... \
              </span></h3>'))
fig = plt.figure(figsize=(3, 3), dpi=300)

ax = fig.add_subplot(1, 1, 1, projection=ccrs.OSGB())

ax.imshow(island_raster_image,
          origin='upper',
          extent=display_extents,
          transform=ccrs.OSGB(),
          zorder=0)

ax.imshow(heights_array_clipped,
          origin='upper',
          alpha=0.35,
          cmap=cm.jet,
          extent=display_extents,
          transform=ccrs.OSGB(),
          zorder=1)

ax.imshow(legend_img,
          origin='upper',
          extent=legend_extents,
          transform=ccrs.OSGB(),
          zorder=10)

ax.imshow(geotag_img,
          origin='upper',
          extent=geotag_extents,
          transform=ccrs.OSGB(),
          zorder=7)

ax.imshow(geotag_destination_img,
          origin='upper',
          extent=geotag_destination_extents,
          transform=ccrs.OSGB(),
          zorder=7)

shortest_path_gdf.plot(ax=ax,
                       edgecolor='black',
                       linewidth=0.7,
                       linestyle='dashed',
                       transform=ccrs.OSGB(),
                       zorder=6)

off_road_route.plot(ax=ax,
                    edgecolor='black',
                    linewidth=0.7,
                    linestyle='dashed',
                    transform=ccrs.OSGB(),
                    zorder=6)


ax.set_extent(display_extents, crs=ccrs.OSGB())
clear_output()
start_point = ("<h2><center>" + "The point you entered is at " +
               str(int(point[0])) + ", " + str(int(point[1])) +
               "</center></h2>")
end_point = ("<h2><center>" + "Your destination is at " +
             str(int(highest_point[0])) + ", " + str(int(highest_point[1])) +
             "</center></h2>")
display(HTML(start_point))
display(HTML(end_point))
plt.show()


**BS** The program includes several small improvments like <br> the display of loading messages and markdown the displayed text.<br>  

## 